Para realizar a **análise de literatura**, foi desenvolvida uma função com o objetivo de realizar a pesquisa de bibliografia na base de dados NCBI através do BioPython. Esta função oferece uma abordagem flexível uma vez que permite a adaptação de parâmetros conforme os objetivos desejados.

A função escrita recebe um email e um termo de pesquisa como entrada, depois, oferece a capacidade de selecionar a base de dados desejada conforme o leque de bases de dados disponíveis. Por exemplo, podemos selecionar:

- pubmed
- gene
- nuccore
- protein
- ...

Além disso, o número de resultados que integram o output pode ser maleável, tal como o tipo de informação a ser extraído (como abstracts ou sequências genéticas) e também o formato do output. 

### Função **pesquisa_ncbi**

In [1]:
def pesquisa_ncbi(email, term, db = 'pubmed', retmax = 10, rettype = 'abstract', retmode = 'text', display = 'Y' , save = 'N', ext='txt'):
    '''

    Desenvolvido por Rui Sousa

    NOTA - PARA USAR RETMODE XML TÊM DE SER DECLARADA UMA NOVA VARIÁVEL POIS A FUNÇÃO AINDA NÃO CRIA FICHEIROS XML
    
    Função para a pesquisa de bibliografia na base de dados NCBI utilizando BioPython
    Esta função contempla um conjunto de parâmetros para seleção de base de dados e outputs desejado adaptando-se
    ao objetivo da pesquisa. 

    Por defeito a função pesquisa a base de dados pubmed e capta os 10 primeiros resultados para uma pesquisa
    generalizada. 

    Tem a opção de gerar um ficheiro de texto com os resultados da pesquisa de artigos e respetivos abstracts.

    No entanto a função pode ser especializada para uma pesquisa em base de dados de genes para captar ficheiros
    GenBank e FASTA para posterior análise com as funções específicas que estamos a desenvolver.

    Parâmetros
    ----------

    email : str
        email a ser utilizado pelo módulo Entrez para  pesquisa (e.g. exempl@mail.com)

    term : str
        termo a ser utilizado na pesquisa (e.g. 'gene-123 functions')

    db  : str
        base de dados onde irá ser realizada a pesquisa. Por defeito é utilizada a 'pubmed'
        Para efeitos deste trabalho iremos também utilizar: 'gene', 'nuccore' e 'protein'
        Outras opções disponíveis podem ser encontradas aqui: https://www.ncbi.nlm.nih.gov/books/NBK25497/table/chapter2.T._entrez_unique_identifiers_ui/?report=objectonly

    retmax : int
        define o número de resultados que queremos retirar para o output final (10 por defeito)

    rettype : str
        define o tipo de resultado que pretendemos retirar da base de dados ('abstract' por defeito)
        pode ser mudado para 'gb' para base de dados 'gene' ou 'fasta' para base de dados de nucleotidos ou proteina (sequencias)
        
    retmode : str
        define o retmode para a captura de resultados da pesquisa na base de dados. Por defeito 
        o retmode é text mas pode ser adaptado para 'xml' quando é pretendido obter um ficheiro xml (e.g base de dados nucleotide, ou genbank)

    display : str
        define se é pretendido o output da função legível (útil para quando se quer refinar a pesquisa antes/sem gravar o ficheiro)
        por defeito apresenta o resultado em formato legível
    
    save : str
        define se é pretendido gravar ou não o ficheiro de resultados na pasta 'lit-search-output'
        por defeito este parâmetro está inactivo

            
    
    '''
    # TODO criar ficheiros XML

    # Importação de módulos
    from Bio import Entrez
    import os

    # Definimos o email para os fins de pesquisa
    Entrez.email = email
        
    # Secção egquery que pode ser skipable

    handle = Entrez.egquery(term=term)
    egq_res = Entrez.read(handle)

    if rettype == 'fasta' or rettype == 'gb': retmax = 1

    for _ in egq_res['eGQueryResult']:
        if db in _.values():
            if retmax > 1:
                print('Encontrados {} resultados em {}. Irão ser processados {} resultados.\n'.format(_['Count'],db,retmax))
            else:
                print('Encontrados {} resultados em {}. Irá ser processado 1 resultado.\n'.format(_['Count'],db))

    # Sacar as IDs
    handle      = Entrez.esearch(db=db,term=term, retmode=retmode, retmax=retmax)
    esearch_res = Entrez.read(handle)

    lista_ids = esearch_res['IdList']
    
    # Transfere a informação
    handle = Entrez.efetch(db=db,id=','.join(lista_ids),rettype=rettype,retmode=retmode)
    fetch_res = handle.read()
    
    if display.upper() == 'Y':
        print(fetch_res,sep='\n')


    # Verifica a condição de gravação de output como ficheiro de texto
    
    if retmode == 'xml': save = 'n'
    
    if save.upper() == 'Y':  

        # Definimos a extensão em função da base de dados e retmode
        

        filename = term + type_handler(rettype,retmode)

        if os.path.isfile(filename): 
            print('Ficheiro já existe. Não será criado novo ficheiro.')
            return fetch_res

        # Gera o ficheiro           
        guardar_ficheiro(filename, fetch_res)


    handle.close()

    return fetch_res


# SUBFUNÇÕES

def guardar_ficheiro(ficheiro, content):

    '''
    Função que recebe um nome de ficheiro (ficheiro) e carrega o conteúdo 
    da variável content para esse ficheiro.
    '''    
    with open(ficheiro, 'w', encoding = 'utf-8') as _:
        _.write(content)

    print(f'{ficheiro} guardado com sucesso.')

def type_handler(rettype,retmode):

    '''
    Função auxiliar da função pesquisa NCBI que permite
    controlar o tipo de ficheiro de output da função
    '''

    if rettype == 'gb':
        ext = '.gbk'
        
    elif rettype == 'fasta':
        ext = '.fasta'
        
    elif retmode == 'xml':
        ext = '.xml'

    else: ext = '.txt'

    return ext

### Função **pesquisa_ncbi** aplicada

Passando agora à utilização da função com os parâmetros pretendidos. Foi definido o email e o termo de pesquisa pretendido de forma a diminuir o espetro de resultados

In [3]:
email = 'pg52648@alunos.uminho.pt'
termo = 'hla-dqa1'

In [4]:
pesquisa_ncbi(email,termo)

Encontrados 2011 resultados em pubmed. Irão ser processados 5 resultados.

1. Clin Immunol. 2024 Jan 8:109897. doi: 10.1016/j.clim.2024.109897. Online ahead
 of print.

The genome-wide association study of serum IgE levels demonstrated the shared 
genetic background in allergic diseases.

Lu HF(1), Chou CH(2), Lin YJ(3), Uchiyama S(4), Terao C(5), Wang YW(6), Yang 
JS(6), Liu TY(6), Wong HS(7), Chen SC(7), Tsai FJ(8).

Author information:
(1)Department of Medical Research, China Medical University Hospital, Taichung 
404327, Taiwan; Laboratory for Statistical and Translational Genetics, RIKEN 
Center for Integrative Medical Sciences, Yokohama, Japan; Genetic Center, 
Department of Medical Research, China Medical University Hospital, Taichung, 
Taiwan.
(2)PhD Program for Health Science and Industry, College of Health Care, China 
Medical University, Taichung, Taiwan; Department of Health Services 
Administration, China Medical University, Taichung, Taiwan; Department of 
Medical Researc

"1. Clin Immunol. 2024 Jan 8:109897. doi: 10.1016/j.clim.2024.109897. Online ahead\n of print.\n\nThe genome-wide association study of serum IgE levels demonstrated the shared \ngenetic background in allergic diseases.\n\nLu HF(1), Chou CH(2), Lin YJ(3), Uchiyama S(4), Terao C(5), Wang YW(6), Yang \nJS(6), Liu TY(6), Wong HS(7), Chen SC(7), Tsai FJ(8).\n\nAuthor information:\n(1)Department of Medical Research, China Medical University Hospital, Taichung \n404327, Taiwan; Laboratory for Statistical and Translational Genetics, RIKEN \nCenter for Integrative Medical Sciences, Yokohama, Japan; Genetic Center, \nDepartment of Medical Research, China Medical University Hospital, Taichung, \nTaiwan.\n(2)PhD Program for Health Science and Industry, College of Health Care, China \nMedical University, Taichung, Taiwan; Department of Health Services \nAdministration, China Medical University, Taichung, Taiwan; Department of \nMedical Research, China Medical University Hospital, Taichung 404327, T

Como podemos observar, foram processados processados somente 5 resultados dos 2011 resultados encontrados. 

O primeiro resultado apresentado é um artigo intitulado de "The genome-wide association study of serum IgE levels demonstrated the shared 
genetic background in allergic diseases", publicado em janeiro de 2024 na revista Clinical Immunology. O estudo foi conduzido por 8 autores que se encontram listados acima em conjunto com informação relevante sobre os mesmos.

No final do output podemos observar que tanto o DOI e o PMID são fornecidos. 

Podemos concluir que o output da função **pesquisa_ncbi** permite a extração de informações relevantes na análise dos genes a nível da análise de artigos científicos disponíveis em diversas bases de dados. 

### Flexibilidade da função pesquisa_ncbi

Como foi descrito anteriormente, é possível adaptar os parâmetros da função de forma a atingir os objetivos esperados. Desta forma, é possível:

- **Manipular quantidade de resultados processados**

Podemos manipular a quantidade de resultados apresentados modificando o número do parâmetro "retmax". Na função seguinte foi alterado o parâmetro "retmax" para "1" de modo a ser processo um único resultado.

In [5]:
pesquisa_ncbi(email,termo,retmax=1)

Encontrados 2011 resultados em pubmed. Irá ser processado 1 resultado.

1. Clin Immunol. 2024 Jan 8:109897. doi: 10.1016/j.clim.2024.109897. Online ahead
 of print.

The genome-wide association study of serum IgE levels demonstrated the shared 
genetic background in allergic diseases.

Lu HF(1), Chou CH(2), Lin YJ(3), Uchiyama S(4), Terao C(5), Wang YW(6), Yang 
JS(6), Liu TY(6), Wong HS(7), Chen SC(7), Tsai FJ(8).

Author information:
(1)Department of Medical Research, China Medical University Hospital, Taichung 
404327, Taiwan; Laboratory for Statistical and Translational Genetics, RIKEN 
Center for Integrative Medical Sciences, Yokohama, Japan; Genetic Center, 
Department of Medical Research, China Medical University Hospital, Taichung, 
Taiwan.
(2)PhD Program for Health Science and Industry, College of Health Care, China 
Medical University, Taichung, Taiwan; Department of Health Services 
Administration, China Medical University, Taichung, Taiwan; Department of 
Medical Research, 

'1. Clin Immunol. 2024 Jan 8:109897. doi: 10.1016/j.clim.2024.109897. Online ahead\n of print.\n\nThe genome-wide association study of serum IgE levels demonstrated the shared \ngenetic background in allergic diseases.\n\nLu HF(1), Chou CH(2), Lin YJ(3), Uchiyama S(4), Terao C(5), Wang YW(6), Yang \nJS(6), Liu TY(6), Wong HS(7), Chen SC(7), Tsai FJ(8).\n\nAuthor information:\n(1)Department of Medical Research, China Medical University Hospital, Taichung \n404327, Taiwan; Laboratory for Statistical and Translational Genetics, RIKEN \nCenter for Integrative Medical Sciences, Yokohama, Japan; Genetic Center, \nDepartment of Medical Research, China Medical University Hospital, Taichung, \nTaiwan.\n(2)PhD Program for Health Science and Industry, College of Health Care, China \nMedical University, Taichung, Taiwan; Department of Health Services \nAdministration, China Medical University, Taichung, Taiwan; Department of \nMedical Research, China Medical University Hospital, Taichung 404327, T

Podemos observar que, dos 2011 resultados obtidos, apenas 1 foi processado.

Para além da alteração da quantidade de resultados processados é igualmente possível realizar as seguintes manipulações da função:

-  **Manipular o display do output**

-  **Manipulação da gravação do ficheiro**

A manipulação do display do output é alcançada através da alteração do parâmetro "display" e o modo de gravação dos resultados processados é feita através da alteração do parâmetro "save".

In [6]:
# Pesquisa com output, sem display legível e permitindo a gravação dos resultados processados
pesquisa_ncbi(email,termo,save='y',display='n')

Encontrados 2011 resultados em pubmed. Irão ser processados 5 resultados.

Ficheiro já existe. Não será criado novo ficheiro.
Ficheiro gravado.
Concluído.


"1. Clin Immunol. 2024 Jan 8:109897. doi: 10.1016/j.clim.2024.109897. Online ahead\n of print.\n\nThe genome-wide association study of serum IgE levels demonstrated the shared \ngenetic background in allergic diseases.\n\nLu HF(1), Chou CH(2), Lin YJ(3), Uchiyama S(4), Terao C(5), Wang YW(6), Yang \nJS(6), Liu TY(6), Wong HS(7), Chen SC(7), Tsai FJ(8).\n\nAuthor information:\n(1)Department of Medical Research, China Medical University Hospital, Taichung \n404327, Taiwan; Laboratory for Statistical and Translational Genetics, RIKEN \nCenter for Integrative Medical Sciences, Yokohama, Japan; Genetic Center, \nDepartment of Medical Research, China Medical University Hospital, Taichung, \nTaiwan.\n(2)PhD Program for Health Science and Industry, College of Health Care, China \nMedical University, Taichung, Taiwan; Department of Health Services \nAdministration, China Medical University, Taichung, Taiwan; Department of \nMedical Research, China Medical University Hospital, Taichung 404327, T

Como podemos observar pelo resultado apresentado, não ocorreu o display legível dos resultados e o ficheiro foi gravado.

Também é possível realizar a seguinte manipulação da função:

- **Manipular bases de dados onde a pesquisa é efetuada**

O parâmetro "db" pode ser alterado de forma a alterar a base de dados onde a pesquisa é realizada. No seguinte exemplo podemos observar que a base de dados selecionada para efeitos de pesquisa dos resultados foi a "nuccore".

In [7]:
# Pesquisa em base de dados de nucleotide/protein/gene
pesquisa_ncbi(email,termo,db='nuccore')

Encontrados 1769 resultados em nuccore. Irão ser processados 5 resultados.

LOCUS       OY986031                7342 bp    DNA     linear   PRI 27-DEC-2023
DEFINITION  Homo sapiens HLA-DQA1 gene for MHC class II antigen, allele
            HLA-DQA1*02:01:01:NEW.
ACCESSION   OY986031
VERSION     OY986031.1
DBLINK      BioProject: PRJEB71396
KEYWORDS    .
SOURCE      Homo sapiens (human)
  ORGANISM  Homo sapiens
            Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi;
            Mammalia; Eutheria; Euarchontoglires; Primates; Haplorrhini;
            Catarrhini; Hominidae; Homo.
REFERENCE   1
  AUTHORS   French,A.
  TITLE     Direct Submission
  JOURNAL   Submitted (19-DEC-2023) Anthony Nolan Research Institute, Anthony
            Nolan Research Institute, 77C Fleet Road, Hampstead Heath, United
            Kingdom
FEATURES             Location/Qualifiers
     source          1..7342
                     /organism="Homo sapiens"
                     /mol_type="geno

'LOCUS       OY986031                7342 bp    DNA     linear   PRI 27-DEC-2023\nDEFINITION  Homo sapiens HLA-DQA1 gene for MHC class II antigen, allele\n            HLA-DQA1*02:01:01:NEW.\nACCESSION   OY986031\nVERSION     OY986031.1\nDBLINK      BioProject: PRJEB71396\nKEYWORDS    .\nSOURCE      Homo sapiens (human)\n  ORGANISM  Homo sapiens\n            Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi;\n            Mammalia; Eutheria; Euarchontoglires; Primates; Haplorrhini;\n            Catarrhini; Hominidae; Homo.\nREFERENCE   1\n  AUTHORS   French,A.\n  TITLE     Direct Submission\n  JOURNAL   Submitted (19-DEC-2023) Anthony Nolan Research Institute, Anthony\n            Nolan Research Institute, 77C Fleet Road, Hampstead Heath, United\n            Kingdom\nFEATURES             Location/Qualifiers\n     source          1..7342\n                     /organism="Homo sapiens"\n                     /mol_type="genomic DNA"\n                     /db_xref="taxon:9606"\n

A execução da função pesquisa_ncbi para a base de dados 'nuccore' resultou na identificação de 1769 entradas relacionadas. Destas, foram processados os primeiros 5 resultados.

O primeiro resultado apresentado é o registo com o Accession Number OY986031, correspondente a um alelo do gene do antígeno de classe II do MHC em Homo sapiens (HLA-DQA1).

O output dos resultados da pesquisa na base de dados "nuccore" foi em formato "genbank". Nos resultados são incorporados detalhes característicos desse tipo de formato incluindo a origem da sequência, características e a própria sequência, sendo essas informações enriquecedoras no que toca ao aprofundamento da análise do gene em questão. 


### Transferência do ficheiro correspondente ao gene de interesse

De forma a realizar a transferência do ficheiro correspondente ao gene de interesse, em primeiro lugar, foi efetuada uma pesquisa na base de dados "gene" de forma a identificar o ID do gene de interesse. 

In [11]:
pesquisa_ncbi(email,"HLA-DQA1",db='gene',retmax=15)

Encontrados 189 resultados em gene. Irão ser processados 15 resultados.


1. HLA-DRB1
Official Symbol: HLA-DRB1 and Name: major histocompatibility complex, class II, DR beta 1 [Homo sapiens (human)]
Other Aliases: DRB1, HLA-DR1B, HLA-DRB, SS1
Other Designations: major histocompatibility complex, class II, DR beta 1 precursor; HLA class II histocompatibility antigen, DR-1 beta chain; MHC class II HLA-DR beta 1 chain; human leucocyte antigen DRB1; lymphocyte antigen DRB1
Chromosome: 6; Location: 6p21.32
Annotation: Chromosome 6 NC_000006.12 (32578775..32589848, complement)
MIM: 142857
ID: 3123

2. env
Envelope surface glycoprotein gp160, precursor [Human immunodeficiency virus 1]
Other Aliases: HIV1gp8
Other Designations: Envelope surface glycoprotein gp160, precursor; Envelope surface glycoprotein gp120; Envelope transmembrane domain; hypothetical protein
Annotation:  NC_001802.1 (5771..8341)
ID: 155971

3. TERT
Official Symbol: TERT and Name: telomerase reverse transcriptase [Homo sapi

'\n1. HLA-DRB1\nOfficial Symbol: HLA-DRB1 and Name: major histocompatibility complex, class II, DR beta 1 [Homo sapiens (human)]\nOther Aliases: DRB1, HLA-DR1B, HLA-DRB, SS1\nOther Designations: major histocompatibility complex, class II, DR beta 1 precursor; HLA class II histocompatibility antigen, DR-1 beta chain; MHC class II HLA-DR beta 1 chain; human leucocyte antigen DRB1; lymphocyte antigen DRB1\nChromosome: 6; Location: 6p21.32\nAnnotation: Chromosome 6 NC_000006.12 (32578775..32589848, complement)\nMIM: 142857\nID: 3123\n\n2. env\nEnvelope surface glycoprotein gp160, precursor [Human immunodeficiency virus 1]\nOther Aliases: HIV1gp8\nOther Designations: Envelope surface glycoprotein gp160, precursor; Envelope surface glycoprotein gp120; Envelope transmembrane domain; hypothetical protein\nAnnotation:  NC_001802.1 (5771..8341)\nID: 155971\n\n3. TERT\nOfficial Symbol: TERT and Name: telomerase reverse transcriptase [Homo sapiens (human)]\nOther Aliases: CMM9, DKCA2, DKCB4, EST2,

Através da análise do output da função anterior, podemos observar que o ID correspondente ao gene de interesse é o "3117". Assim, torna-se possível proceder à gravação do ficheiro de interesse. 

Utilizando a função **pesquisa_ncbi** e colocando os seguintes parâmetros na função:

In [13]:
pesquisa_ncbi(email,'3117',db='gene',rettype='gb',retmax=1,save='y',display='n')

Encontrados 1 resultados em gene. Irá ser processado 1 resultado.

Ficheiro gravado.
Concluído.


'\n1. HLA-DQA1\nOfficial Symbol: HLA-DQA1 and Name: major histocompatibility complex, class II, DQ alpha 1 [Homo sapiens (human)]\nOther Aliases: CELIAC1, DQ-A1, DQA1, HLA-DQA*, HLA-DQB1, HLA-DQA1\nOther Designations: HLA class II histocompatibility antigen, DQ alpha 1 chain; DC-1 alpha chain; DC-alpha; HLA class II histocompatibility antigen DQ alpha chain; HLA-DCA; MHC HLA-DQ alpha; MHC class II DQ alpha chain; MHC class II DQA1; MHC class II HLA-DQ-alpha-1; MHC class II antigen DQA1; MHC class II protein\nChromosome: 6; Location: 6p21.32\nAnnotation: Chromosome 6 NC_000006.12 (32637406..32655272)\nMIM: 146880\nID: 3117\n\n'

Foi gravado com sucesso o ficheiro pretendido.

### Transferência do ficheiro correspondente à proteína de interesse

Passando agora para a identificação e preparação de dados relacionados à proteína codificada pelo gene HLA-DQA1 para análises subsequentes. Utilizando a função pesquisa_ncbi, direcionamos a pesquisa para a base de dados 'protein' da NCBI, procurando recuperar informações específicas sobre a proteína de interesse.

In [12]:
pesquisa_ncbi(email,'DQA1_HUMAN',db='protein')

Encontrados 1 resultados em protein. Irão ser processados 5 resultados.

LOCUS       DQA1_HUMAN               254 aa            linear   PRI 08-NOV-2023
DEFINITION  RecName: Full=HLA class II histocompatibility antigen, DQ alpha 1
            chain; AltName: Full=DC-1 alpha chain; AltName: Full=DC-alpha;
            AltName: Full=HLA-DCA; AltName: Full=MHC class II DQA1; Flags:
            Precursor.
ACCESSION   P01909
VERSION     P01909.1
DBSOURCE    UniProtKB: locus DQA1_HUMAN, accession P01909;
            class: standard.
            extra
            accessions:O19630,O19706,P01907,P01908,P04225,P04226,P05536,P79553,
            Q06751,Q29876,Q29994,Q2Q6Y6,Q2Q6Y7,Q2Q6Y8,Q2WCM3,Q30064,Q30067,
            Q30068,Q30070,Q30071,Q30072,Q30073,Q30086,Q30101,Q5Y7D5,Q5Y7F5,
            Q6ICU6,Q6PR46,Q6QDB1,Q860W2,Q860W4,Q9BD37,Q9TPM3,Q9UM31
            created: Jul 21, 1986.
            sequence updated: Jul 21, 1986.
            annotation updated: Nov 8, 2023.
            xrefs: X00033.

'LOCUS       DQA1_HUMAN               254 aa            linear   PRI 08-NOV-2023\nDEFINITION  RecName: Full=HLA class II histocompatibility antigen, DQ alpha 1\n            chain; AltName: Full=DC-1 alpha chain; AltName: Full=DC-alpha;\n            AltName: Full=HLA-DCA; AltName: Full=MHC class II DQA1; Flags:\n            Precursor.\nACCESSION   P01909\nVERSION     P01909.1\nDBSOURCE    UniProtKB: locus DQA1_HUMAN, accession P01909;\n            class: standard.\n            extra\n            accessions:O19630,O19706,P01907,P01908,P04225,P04226,P05536,P79553,\n            Q06751,Q29876,Q29994,Q2Q6Y6,Q2Q6Y7,Q2Q6Y8,Q2WCM3,Q30064,Q30067,\n            Q30068,Q30070,Q30071,Q30072,Q30073,Q30086,Q30101,Q5Y7D5,Q5Y7F5,\n            Q6ICU6,Q6PR46,Q6QDB1,Q860W2,Q860W4,Q9BD37,Q9TPM3,Q9UM31\n            created: Jul 21, 1986.\n            sequence updated: Jul 21, 1986.\n            annotation updated: Nov 8, 2023.\n            xrefs: X00033.1, CAA24917.1, X00370.1, X00452.1, CAA25141.1,\n       

Como conseguimos obter o Accession Number da proteína codificada pelo gene de interesse, procedemos à gravação do ficheiro da mesma.

In [13]:
pesquisa_ncbi(email,'P01909',db='protein',retmax=1,rettype='fasta',save='Y',display='n')

Encontrados 10 resultados em protein. Irá ser processado 1 resultado.

Ficheiro já existe. Não será criado novo ficheiro.
Ficheiro gravado.
Concluído.


'>sp|P01909.1|DQA1_HUMAN RecName: Full=HLA class II histocompatibility antigen, DQ alpha 1 chain; AltName: Full=DC-1 alpha chain; AltName: Full=DC-alpha; AltName: Full=HLA-DCA; AltName: Full=MHC class II DQA1; Flags: Precursor\nMILNKALMLGALALTTVMSPCGGEDIVADHVASYGVNLYQSYGPSGQYTHEFDGDEQFYVDLGRKETVWC\nLPVLRQFRFDPQFALTNIAVLKHNLNSLIKRSNSTAATNEVPEVTVFSKSPVTLGQPNILICLVDNIFPP\nVVNITWLSNGHSVTEGVSETSFLSKSDHSFFKISYLTLLPSAEESYDCKVEHWGLDKPLLKHWEPEIPAP\nMSELTETVVCALGLSVGLVGIVVGTVFIIRGLRSVGASRHQGPL\n\n'